In [1]:
map_name = 'k_near_map'
# --- Generate Map --- #
import folium

center = [33.748997, -84.387985]
m = folium.Map(center, zoom_start=10)
m.save(f'./outputs/maps/{map_name}.html')

m = folium.Map(
    location=[33.748997, -84.387985],
    tiles='Stamen Toner',
    zoom_start=10)

In [2]:
import pandas as pd
import geopandas
from shapely.geometry import shape,mapping, Point, Polygon, MultiPolygon
import matplotlib.pyplot as plt
# import geoplot
from project_tools import load_obj, interact, save_obj

def CreatePointMap(df, Point_name):
    # cleaning
    df[['Latitude','Longitude']] = df[['Latitude','Longitude']].apply(pd.to_numeric, errors='coerce')
    # --- Converts df type to GeoDataFrame --- #
    pointDF = df
    df['coordinates'] = df[['Latitude', 'Longitude']].values.tolist()
    df['coordinates'] = df['coordinates'].apply(Point)
    df = geopandas.GeoDataFrame(df, geometry='coordinates')
    df = df.dropna(subset=[Point_name,'Latitude', 'Longitude','coordinates'])
    return df

def GeneratePointMap(Names , Latitudes, Longitudes, color, radius):
    for i, Name in enumerate(Names.iterrows()):
        folium.CircleMarker(
            location=[Latitudes.iloc[i,0],Longitudes.iloc[i,0]],
            tooltip = str(Names.iloc[i,0]),
            radius= radius,
            popup=str(Names.iloc[i,0]),
            color=color,
            fill=True,
            fill_color=color 
        ).add_to(m)
        
def covertname(x):
    try:
        x=inv_matcher[x]
    except KeyError:
        try:
            x=map_[x]
        except KeyError:
            x=0
    return int(x)

def plotfromdf(df, name, color='blue', size=3):
    df = CreatePointMap(df, name)
    GeneratePointMap(df.loc[:,[name]], df.loc[:,['Latitude']],df.loc[:,['Longitude']], color, radius=size)

In [3]:
# map_ is a dict converting names to station numbers
map_=server_info = load_obj('map')
del map_['would_not_load']
del map_['loaded_not_a_name']
map_df = pd.DataFrame(map_, index=['vds_station_number']).T
# info is a dict with extended station info eg lat long
# mostly all formatting
info=server_info = load_obj('info')
info_df = pd.DataFrame(info).T
names = pd.read_csv('support//inventory_xpath.csv',usecols = [0]).names # col names
info_df.columns = names
info_df['names'] = info_df.index
info_df.index = pd.RangeIndex(len(info_df.index))

vds_info = pd.read_csv('data/vds/location_data/vds_info_total.csv')
matcher = dict(zip(list(vds_info['value']),list(vds_info['description'])))


# remove if not GDOT
delete = []
onlyGDOT = True
for key, val in matcher.items():
    if onlyGDOT:
        if val[0] != 'G':
            delete.append(key)
        else:
            matcher[key] = val.split(':')[0]
    else:
        matcher[key] = val.split(':')[0]
for key in delete:
    del matcher[key]
inv_matcher = {v:k for k, v in matcher.items()}
#convert ID name to number format
info_df['ID2'] = info_df['ID'].map(covertname)
# only the ones being scrapped
info_df = info_df[info_df['ID2']!=0]

# --- plot all GDOT VDS stations --- #
info_df['ID3'] = info_df['ID2'].astype(str)+' | '+info_df['Direction']+' | '+info_df['Roadway_Name']
plotfromdf(info_df, 'ID3', color='blue', size=3)

In [4]:
CCS_info = pd.read_csv("./data/vds/location_data/ccs_info.csv")
CCS_info['fixed_idx'] = pd.RangeIndex(len(CCS_info.index))

# --- plot all CCS stations --- #
CCS_info['fixed_idx_v2'] = CCS_info['fixed_idx'].astype(str)+' | '+CCS_info['description']
plotfromdf(CCS_info, 'fixed_idx_v2', color='red', size=4)

In [5]:
keep_vds = load_obj('keep_vds')
print(len(keep_vds))
selected = info_df[info_df['ID2'].isin(keep_vds)]
print(len(selected))
# selected['dis'] = selected['ID2'].astype(str)+' | '+selected['Direction']+' | '+selected['Roadway_Name']
# --- plot all CCS stations --- #
plotfromdf(selected, 'ID3', color='green', size=6)

240
239


/home/clint/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3065: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
/home/clint/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/home/clint/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See t

In [6]:
m.save(f'./outputs/maps/{map_name}.html')

In [20]:
matched = load_obj('ccs_vds_matched')
selected_CCS = CCS_info[CCS_info['fixed_idx'].isin(matched.keys())]
plotfromdf(selected_CCS, 'fixed_idx', color='purple', size=10)

/home/clint/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3065: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
/home/clint/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/home/clint/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See t

In [10]:
m.save(f'./outputs/maps/{map_name}.html')

In [35]:
# reformat CCS names
def mapCCSnames(name):
    name = name[5:]
    if '#' in name:
        name = name.replace('#','')
    return f'{name[:3]}-{name[3:]}'

CCS_info['standard_name'] = CCS_info['name'].map(mapCCSnames)
newmap = dict(zip(CCS_info['fixed_idx'].tolist(),CCS_info['standard_name'].tolist()))
formatted = {}
for k in matched.keys():
    formatted[newmap[k]]=matched[k]

In [39]:
vds_direction = dict(zip(info_df['ID2'].tolist(),info_df['Direction'].tolist())) # maps vds name to direction (N/W/E/S)

In [40]:
# --- Matched CCS to VDS --- #
# save_obj(formatted, 'VDS_CCS_MATCHED!')
# save_obj(vds_direction, 'vds_direction')